## Importing required packages

In [1]:
import requests
import lxml.html as lh
import pandas as pd

## Scraping Table from the URL

In [2]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

In [3]:
#Create a handle, page, to handle the contents of the website
page = requests.get(url)
#Store the contents of the website under doc
doc = lh.fromstring(page.content)
#Parse data that are stored between <tr>..</tr> of HTML
tr_elements = doc.xpath('//tr')

In [4]:
#Check the length of the first 12 rows
[len(T) for T in tr_elements[:12]]

[3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3]

## Parse table Header

In [5]:
#Create empty list
col=[]
i=0
#For each row, store each first element (header) and an empty list
for t in tr_elements[0]:
    i+=1
    name=t.text_content()
    print('{}:{}'.format(i,name))
    col.append((name,[]))

1:Postcode
2:Borough
3:Neighbourhood



## Creating Pandas Dataframe

In [6]:
#Since out first row is the header, data is stored on the second row onwards
for j in range(1,len(tr_elements)):
    #T is our j'th row
    T=tr_elements[j]
    
    #If row is not of size 10, the //tr data is not from our table 
    if len(T)!=3:
        break
    
    #i is the index of our column
    i=0
    
    #Iterate through each element of the row
    for t in T.iterchildren():
        data=t.text_content() 
        #Check if row is empty
        if i>0:
        #Convert any numerical value to integers
            try:
                data=int(data)
            except:
                pass
        #Append the data to the empty list of the i'th column
        col[i][1].append(data)
        #Increment i for the next column
        i+=1

In [7]:
[len(C) for (title,C) in col]

[288, 288, 288]

In [8]:
Dict={title:column for (title,column) in col}
df=pd.DataFrame(Dict)

In [9]:
df.head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned\n
1,M2A,Not assigned,Not assigned\n
2,M3A,North York,Parkwoods\n
3,M4A,North York,Victoria Village\n
4,M5A,Downtown Toronto,Harbourfront\n


## Cleaning dataframe: removing \n

In [10]:
df.columns = ['Postcode', 'Borough', 'Neighbourhood']

In [11]:
df.Neighbourhood = df.Neighbourhood.str.replace('\n', '')

In [12]:
df.head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


## Removing rows where column Borough is not assigned

In [13]:
df = df[df.Borough != "Not assigned"]

In [14]:
df.head()

,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights


## Replacing Neighbourhood that are not assigned by their corresponding Borough

In [15]:
df[df.Neighbourhood == 'Not assigned']

,Postcode,Borough,Neighbourhood
8,M7A,Queen's Park,Not assigned


In [16]:
df.Neighbourhood.replace("Not assigned", df[df.Neighbourhood == 'Not assigned'].Borough, inplace=True)

In [17]:
df.head()

,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights


## Resetting the index: Final Dataset

In [97]:
df.drop(df.index[136], inplace=True)

In [98]:
df = df.reset_index(drop=True)

In [99]:
df.head()

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights


## Custom function to geocode postcode

In [102]:
import time

In [103]:
def geocode(postalcode):
    API_KEY = 'dc42f8f1757c17'
    url = 'https://us1.locationiq.com/v1/search.php'
    params = {'key': API_KEY, 'format':'json', 'postalcode':postalcode}
    r = requests.get(url, params=params)
    result = r.json()
    time.sleep(1) # Only 60 requests are allowed per minute so delaying each request by sec 
    print((result[0]['lat'], result[0]['lon']))
    return (result[0]['lat'], result[0]['lon'])

## Generating column for Latitude

In [107]:
df['Latitude'] = df['Postcode'].apply(lambda x: geocode(x)[0])

('43.7545', '-79.33')
('43.7276', '-79.3148')
('43.6555', '-79.3626')
('43.6555', '-79.3626')
('43.7223', '-79.4504')
('43.7223', '-79.4504')
('43.6641', '-79.3889')
('43.6662', '-79.5282')
('43.8113', '-79.193')
('43.8113', '-79.193')
('43.745', '-79.359')
('43.7063', '-79.3094')
('43.7063', '-79.3094')
('43.6572', '-79.3783')
('43.6572', '-79.3783')
('43.7081', '-79.4479')
('43.6505', '-79.5517')
('43.6505', '-79.5517')
('43.6505', '-79.5517')
('43.6505', '-79.5517')
('43.6505', '-79.5517')
('43.7878', '-79.1564')
('43.7878', '-79.1564')
('43.7878', '-79.1564')
('43.7334', '-79.3329')
('43.7334', '-79.3329')
('43.6913', '-79.3116')
('43.6513', '-79.3756')
('43.6915', '-79.4307')
('43.6437', '-79.5767')
('43.6437', '-79.5767')
('43.6437', '-79.5767')
('43.6437', '-79.5767')
('43.7678', '-79.1866')
('43.7678', '-79.1866')
('43.7678', '-79.1866')
('43.6784', '-79.2941')
('43.6456', '-79.3754')
('43.6889', '-79.4507')
('43.7712', '-79.2144')
('43.7124', '-79.3644')
('43.6564', '-79.386')

In [108]:
df.head()

,Postcode,Borough,Neighbourhood,Latitude
0,M3A,North York,Parkwoods,43.7545
1,M4A,North York,Victoria Village,43.7276
2,M5A,Downtown Toronto,Harbourfront,43.6555
3,M5A,Downtown Toronto,Regent Park,43.6555
4,M6A,North York,Lawrence Heights,43.7223


## Generating column for longitudes

In [109]:
df['Longitude'] = df['Postcode'].apply(lambda x: geocode(x)[1])

('43.7545', '-79.33')
('43.7276', '-79.3148')
('43.6555', '-79.3626')
('43.6555', '-79.3626')
('43.7223', '-79.4504')
('43.7223', '-79.4504')
('43.6641', '-79.3889')
('43.6662', '-79.5282')
('43.8113', '-79.193')
('43.8113', '-79.193')
('43.745', '-79.359')
('43.7063', '-79.3094')
('43.7063', '-79.3094')
('43.6572', '-79.3783')
('43.6572', '-79.3783')
('43.7081', '-79.4479')
('43.6505', '-79.5517')
('43.6505', '-79.5517')
('43.6505', '-79.5517')
('43.6505', '-79.5517')
('43.6505', '-79.5517')
('43.7878', '-79.1564')
('43.7878', '-79.1564')
('43.7878', '-79.1564')
('43.7334', '-79.3329')
('43.7334', '-79.3329')
('43.6913', '-79.3116')
('43.6513', '-79.3756')
('43.6915', '-79.4307')
('43.6437', '-79.5767')
('43.6437', '-79.5767')
('43.6437', '-79.5767')
('43.6437', '-79.5767')
('43.7678', '-79.1866')
('43.7678', '-79.1866')
('43.7678', '-79.1866')
('43.6784', '-79.2941')
('43.6456', '-79.3754')
('43.6889', '-79.4507')
('43.7712', '-79.2144')
('43.7124', '-79.3644')
('43.6564', '-79.386')

In [110]:
df.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.7545,-79.33
1,M4A,North York,Victoria Village,43.7276,-79.3148
2,M5A,Downtown Toronto,Harbourfront,43.6555,-79.3626
3,M5A,Downtown Toronto,Regent Park,43.6555,-79.3626
4,M6A,North York,Lawrence Heights,43.7223,-79.4504


## Final Dataset

In [111]:
df

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.7545,-79.33
1,M4A,North York,Victoria Village,43.7276,-79.3148
2,M5A,Downtown Toronto,Harbourfront,43.6555,-79.3626
3,M5A,Downtown Toronto,Regent Park,43.6555,-79.3626
4,M6A,North York,Lawrence Heights,43.7223,-79.4504
5,M6A,North York,Lawrence Manor,43.7223,-79.4504
6,M7A,Queen's Park,Queen's Park,43.6641,-79.3889
7,M9A,Etobicoke,Islington Avenue,43.6662,-79.5282
8,M1B,Scarborough,Rouge,43.8113,-79.193
9,M1B,Scarborough,Malvern,43.8113,-79.193


In [115]:
df.to_csv("toronto.csv", index=False)